In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.Session(config=config)

import keras
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import *
from keras.backend import tf as ktf
from keras.callbacks import EarlyStopping
from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import clear_output

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import csv
import os
from PIL import Image
%matplotlib inline
from jupyterthemes import jtplot

jtplot.style()

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(224, 224),
        class_mode='categorical'
)

Found 4750 images belonging to 12 classes.


In [98]:
X_train = np.load('data/preprocessed/X_train.npy')
Y_train = np.load('data/preprocessed/Y_train.npy')
X_test = np.load('data/preprocessed/X_test.npy')
X_train.shape, Y_train.shape, X_test.shape

((4750, 224, 224, 3), (4750, 12), (794, 224, 224, 3))

#### Utility functions

In [100]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1,)
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(' accuracy')


    plt.subplot(1, 2, 2)
    
    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.legend(['train', 'val'], loc='upper left')
    plt.title('loss')
    plt.show()

### CNNs

In [126]:
def seqModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(12))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

In [127]:
cnnmodel = seqModel()

In [134]:
cnnmodel_history = cnnmodel.fit(X_train, Y_train, batch_size=5, epochs=20, validation_split=0.2,
             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

Train on 3800 samples, validate on 950 samples
Epoch 1/20
3800/3800 [==============================] - 28s 7ms/step - loss: 1.8302 - acc: 0.3705 - val_loss: 11.3567 - val_acc: 0.0305
Epoch 2/20
3800/3800 [==============================] - 26s 7ms/step - loss: 1.0903 - acc: 0.6437 - val_loss: 11.1574 - val_acc: 0.0337
Epoch 3/20
3800/3800 [==============================] - 27s 7ms/step - loss: 0.8409 - acc: 0.7303 - val_loss: 15.1315 - val_acc: 0.0442
Epoch 4/20
3800/3800 [==============================] - 27s 7ms/step - loss: 0.6590 - acc: 0.7839 - val_loss: 15.1831 - val_acc: 0.0411: 0s - loss: 0.6590 - acc:
Epoch 5/20
3800/3800 [==============================] - 27s 7ms/step - loss: 0.5612 - acc: 0.8168 - val_loss: 15.2082 - val_acc: 0.0442
Epoch 6/20
3800/3800 [==============================] - 28s 7ms/step - loss: 0.4828 - acc: 0.8418 - val_loss: 15.1921 - val_acc: 0.0453
Epoch 7/20
3800/3800 [==============================] - 27s 7ms/step - loss: 0.4427 - acc: 0.8632 - val_loss: 1

## Transfer Learning

In [97]:
inception_model = InceptionV3(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)

In [99]:
inc_train_bf = inception_model.predict(X_train, verbose=1)
inc_test_bf = inception_model.predict(X_test, verbose=1)

794/794 [==============================] - 5s 6ms/step


In [114]:
def top_model(input_shape):
    input_img = Input(input_shape)
    X = GlobalAveragePooling2D()(input_img)
#     X = Flatten(input_shape=input_shape)(input_img)
    X = Dropout(0.5)(X)   
    
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
        
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
        
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(12, activation='softmax')(X)
    
    model = Model(inputs=input_img, outputs=X)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',#optimizers.SGD(lr=1e-4, momentum=0.9),#'adam',
                 metrics=['accuracy'])
    
    return model

In [118]:
inctop_model = top_model(inc_train_bf.shape[1:])

In [119]:
inc_history = inctop_model.fit(inc_train_bf, Y_train, batch_size=100, epochs=20, validation_split=0.2,
             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

Train on 3800 samples, validate on 950 samples
Epoch 1/20
3800/3800 [==============================] - 6s 1ms/step - loss: 7.7893 - acc: 0.1918 - val_loss: 4.5785 - val_acc: 0.0095
Epoch 2/20
3800/3800 [==============================] - 1s 335us/step - loss: 2.3135 - acc: 0.2495 - val_loss: 3.5075 - val_acc: 0.0126
Epoch 3/20
3800/3800 [==============================] - 1s 328us/step - loss: 1.8984 - acc: 0.2816 - val_loss: 3.6776 - val_acc: 0.0011
Epoch 4/20
3800/3800 [==============================] - 1s 326us/step - loss: 1.7929 - acc: 0.3200 - val_loss: 4.2639 - val_acc: 0.0105
Epoch 5/20
3800/3800 [==============================] - 1s 328us/step - loss: 1.7516 - acc: 0.3221 - val_loss: 4.6062 - val_acc: 0.0189
Epoch 6/20
3800/3800 [==============================] - 1s 335us/step - loss: 1.6763 - acc: 0.3661 - val_loss: 4.4984 - val_acc: 0.0032
Epoch 7/20
3800/3800 [==============================] - 1s 342us/step - loss: 1.6182 - acc: 0.3655 - val_loss: 5.2614 - val_acc: 0.0242
Epo

### Prediction 

In [5]:
test = pd.read_csv('sample_submission.csv')

In [135]:
ind_to_class = {y:x for x,y in train_generator.class_indices.items()}
preds = model.predict(X_test, verbose=1)
sub = pd.DataFrame(test)
sub['species'] = np.argmax(preds, axis=1)
sub['species'] = sub['species'].apply(lambda x: ind_to_class[x])
sub.to_csv('sub.csv', index=False)

ResourceExhaustedError: OOM when allocating tensor with shape[32,3,224,224]
	 [[Node: conv2d_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_conv2d_1_input_0_0/_123, conv2d_1/kernel/read)]]

Caused by op 'conv2d_1/convolution', defined at:
  File "/home/eee/ug/15084015/anaconda3/envs/ML/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/eee/ug/15084015/anaconda3/envs/ML/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/eee/ug/15084015/anaconda3/envs/ML/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/eee/ug/15084015/anaconda3/envs/ML/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/eee/ug/15084015/anaconda3/envs/ML/lib/python3.5/site-pac